In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn import metrics
eps = np.finfo(float).eps

In [2]:
max_epochs = 200
lamb_const = 1e-3
bsize = 442

In [3]:
from sklearn import datasets
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target
# whitening
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

In [4]:
X = X[:, np.newaxis, 2]
# X = X[:, np.newaxis, :]

In [5]:
ni, nd = X.shape
X = np.append(X, np.ones((ni, 1)), axis=1) # append the bias/const term

In [6]:
X.shape

(442, 2)

In [7]:
"""
Init theta with ridge regression
"""
theta = np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T, X) + lamb_const*np.diag(np.ones(nd+1))), X.T), y)

In [8]:
# ind = 0
# loss = 0
# while ind < ni:
#     end = ind + bsize if ind + bsize <= ni else ni
#     bX = X[ind:end]
#     by = y[ind:end]
#     pred = np.matmul(bX, theta)
#     loss += square_loss(pred, by, theta, lamb_const)

#     for j in range(nd + 1): # for each dimension/feature
#         a_j = 2.0 * (bX ** 2).sum(axis=0)
#         y_pred = np.matmul(bX, theta)
#         c_j = 2.0 * (bX * (by - y_pred + theta[j]*bX[:, j])).sum(axis=0)
#         theta[j] = soft_thresholding(c_j/a_j, lamb_const/a_j)

#     ind += bsize

In [9]:
def soft_thresholding(a, delta):
	"""
	eq. (13.56) in Murphy.
	"""
	sign_a = np.sign(a)
	res = sign_a*np.maximum(np.abs(a) - delta, 0)
	return res

def hard_thresholding(c_j, a_j, lamb_const):
	"""
	page.434 in Murphy.
	"""
	if np.abs(c_j) < lamb_const:
		res = 0.0
	else:
		res = c_j / a_j 
	return res

def square_loss(pred, yin, theta, lamb_const):
	"""
	A typical L2 norm aka. square error with an L1 regularization term.
	"""
	y = yin.copy()
	return np.linalg.norm(pred - y, ord=2) + lamb_const * np.linalg.norm(theta, ord=1)

In [10]:
ind = 0
loss = 0
end = ind + bsize if ind + bsize <= ni else ni
bX = X[ind:end]
by = y[ind:end]
pred = np.matmul(bX, theta)
loss += square_loss(pred, by, theta, lamb_const)

In [13]:
for j in range(nd + 1): # for each dimension/feature
    a_j = 2.0 * np.linalg.norm(bX[:, j], ord=2)
    w = theta[j]
    # theta[j] = 0
    # bXi = bX.copy()
    # bXi[:, j] = 0
    y_pred = np.matmul(bX, theta)
    c_j = 2.0 * (np.matmul(bX[:, j].T, by - y_pred + w*bX[:, j])).sum(axis=0)
    if c_j == 0 or a_j == 0:
        theta[j] = w
    else:
        res = soft_thresholding(c_j/a_j, lamb_const/a_j)
        print('[Debug] c_j:{} a_j:{} res:{}'.format(c_j, a_j, res))
        theta[j] = res

ind += bsize

[Debug] c_j:39921.46653808904 a_j:42.047592083257285 res:949.4352366014596
[Debug] c_j:134486.00000000006 a_j:42.04759208325728 res:3198.423318360491


In [12]:
theta

array([ 949.4352366 , 3198.42331836])

In [27]:
loss

66823.73144011518

In [16]:
j = 0

In [17]:
a_j = 2.0 * (bX[:, j] ** 2).sum(axis=0)

In [18]:
a_j.shape

()

In [19]:
y_pred = np.matmul(bX, theta)

In [20]:
y_pred.shape

(16,)

In [24]:
c_j = 2.0 * (np.matmul(by - y_pred + theta[j]*bX[:, j], bX[:, j])).sum(axis=0)

In [23]:
(by - y_pred + theta[j]*bX[:, j]).shape

(16,)

In [25]:
c_j.shape

()

In [26]:
theta.shape

(2,)

In [27]:
c_j

158.56956138970622

In [28]:
a_j

24.976649621377184

In [29]:
lamb_const

0.001

In [33]:
res = soft_thresholding(c_j/a_j, lamb_const/a_j)

In [34]:
res.shape

()

In [35]:
theta[j] = res

In [36]:
res

6.348672211583954

In [37]:
c_j/a_j

6.348712248979488